In [ ]:
import os
import requests

In [ ]:
%run ees_env_var

In [ ]:
# Bronze parameters
urls = {
    'ees_daily_23_24'   : os.path.join(env_paths['reference'], "ees_daily_data_2324.csv"),
    'ees_daily_22_23'   : os.path.join(env_paths['reference'], "ees_daily_data_2223.csv"),
    'ees_daily'         : "https://github.com/dfe-analytical-services/attendance-data-dashboard/raw/main/data/EES_daily_data.csv",
    "ees_weekly"        : "https://github.com/dfe-analytical-services/attendance-data-dashboard/raw/main/data/EES_weekly_data.csv",
    "ees_ytd"           : "https://github.com/dfe-analytical-services/attendance-data-dashboard/raw/main/data/EES_ytd_data.csv",
}

In [ ]:
for key, url in urls.items():
    print(key)

    if url.startswith("https://github.com/dfe-analytical-services/attendance-data-dashboard/raw/main/"):
        # print("\tGithub")
        # Download the CSV file using requests
        response = requests.get(url)
        # Convert response content to a list of lines
        lines = response.text.splitlines()
        # Create an RDD from the list of lines
        rdd = spark.sparkContext.parallelize(lines)
    else:
        # print("\tReference")
        rdd = url
    
    df = spark.read.csv(rdd, header=True, inferSchema=True)
    df.write.csv(os.path.join(env_paths['bronze'], key), header=True, mode='overwrite')
    print(f"\trow count: {df.count()}")
    # df.printSchema()